## <center>Analiza poziomu PM2.5 w afrykańskich miastach</center>
### Zespół:
<ol>
    <li style='font-size: 20px'>Hubert Kłosowski 242424</li>
    <li style='font-size: 20px'>Krzysztof Kolanek 242425</li>
    <li style='font-size: 20px'>Kamil Małecki 242464</li>
</ol>

### Potrzebne importy

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

### Wczytanie danych

In [ ]:
train = pd.read_csv('data\\Train.csv')
test = pd.read_csv('data\\Test.csv')

train.info()

In [ ]:
train.head()

### Rozbicie daty na składowe

In [ ]:
def change_date(df):
    train['date'] = pd.to_datetime(train['date'])
    df['day'] = df['date'].dt.dayofweek.astype(np.int64)
    df['month'] = df['month'].astype(np.int64)
    return df

train = change_date(train)

### Wykres przedstawiający jakość powietrza z krajach afrykańskich

In [ ]:
sns.lineplot(data=train, x='date', y='pm2_5', hue='country')
plt.title('Jakość powietrza z podziałem na kraje')

### Wykres przedstawiający wartość pm2_5 w zarejestrowanych godzinach

In [ ]:
sns.barplot(data=train, x='hour', y='pm2_5', hue='country')
plt.title('Jakość powietrza w poszczególnych godzinach z podziałem na kraje')

### Wykres przedstawiający wartość pm2_5 z zależności od miesięcy i krajów

In [ ]:
sns.barplot(data=train, x='month', y='pm2_5', hue='country')
plt.title('Jakość powietrza w poszczególnych miesiącach z podziałem na kraje')

### Korelacje poszczególnych grup kolumn

In [ ]:
def correlation():
    for index, el in enumerate(starts_with):
        fig, ax = plt.subplots(figsize=(10, 10))
        selected_columns = [col for col in train.columns if col.startswith(el) or col == 'pm2_5']
        sns.heatmap(train[selected_columns].corr(), annot=True, fmt='.2f', cmap='viridis', ax=ax)
        plt.tight_layout()
        plt.show()

train.drop(columns=['id', 'city', 'country', 'site_id', 'date'], inplace=True)
starts_with = train.columns.str.split('_', expand=True).levels[0].to_frame()
starts_with.drop(['month', 'day', 'hour', 'pm2'], inplace=True)
starts_with = starts_with[0].tolist()

correlation()

## <center>Czyszczenie danych</center>

### Wykresy pudełkowe wskazujące wartości odstające dla sulphurdioxide

In [ ]:
def plot_boxplots(column_group='sulphurdioxide'):
    fig, ax = plt.subplots(nrows=3, ncols=3, figsize=(20, 20))
    similar_columns = [el for el in train.columns if el.startswith(column_group)]
    for index, column in enumerate(similar_columns):
        x, y = divmod(index, 3)
        train[column].plot(kind='box', ax=ax[x][y])

plot_boxplots()

### 1. Usunięcie wartości odstających przy użyciu metody zscore

In [ ]:
from scipy.stats import zscore

def del_outliers(dataframe):
    for col in dataframe.columns:
        vec, indexes = zscore(dataframe[col]), []
        for index in range(len(vec)):
            if -3 <= vec[index] >= 3:
                indexes.append(index)
        dataframe.drop(index=indexes, inplace=True)
    return dataframe

# train = del_outliers(train)

### Wykresy przedstawiające ilość NaN w danej kolumnie w zależności od przyjętej jednostki czasu dla sulphurdioxide

In [ ]:
def plot_nans_based_on(date_unit='month', column_group='sulphurdioxide'):
    fig, ax = plt.subplots(nrows=3, ncols=3, figsize=(20, 20))
    similar_columns = [el for el in train.columns if el.startswith(column_group)]
    for index, column in enumerate(similar_columns):
        x, y = divmod(index, 3)
        nans = train[[column, date_unit]].groupby(date_unit).apply(lambda el: el.isna().sum())
        nans.plot(kind='bar', x=date_unit, y=column, ax=ax[x][y])

plot_nans_based_on()

### 2. Uzupełnienie wartości brakujacych

In [ ]:
from sklearn.impute import KNNImputer

def fill_based_on(dataframe, date_unit='month'):  # uzupelnienie NaN
    date_range = dataframe[date_unit].unique()
    knn_imputer = KNNImputer(n_neighbors=10, weights='distance')
    for date in date_range:
        for column in starts_with:
            similar_columns = [el for el in dataframe.columns if el.startswith(column)]  # wybieramy kolumny o podobnym znaczeniu
            df = dataframe.loc[dataframe[date_unit] == date, similar_columns]  # wybieramy fragment df w tej samej jednostce czasu dla tych grup kolumn
            if df.empty is not True:  # jesli zostaly znalezione jakies rekordy
                dataframe.loc[dataframe[date_unit] == date, similar_columns] = knn_imputer.fit_transform(df)
    return dataframe

def prepare_dataframe(dataframe):  # usuwamy kolumny o dużej liczbie wartości NaN
    to_drop = []
    for index, el in enumerate(dataframe.columns):
        if dataframe[el].isnull().sum() / len(dataframe) * 100 >= 61:
            to_drop.append(el)
    dataframe.drop(to_drop, axis=1, inplace=True)
    return dataframe
    
train = prepare_dataframe(train)
train = fill_based_on(train)

train.info()

In [ ]:
train.head()

### <center>Transformacja danych</center>

In [ ]:
from sklearn.preprocessing import StandardScaler

sts = StandardScaler()
sts.fit(train)
train = sts.transform(train)

### Praca z PyTorch
<ol>
    <li>Ustalanie karty graficznej jako urządzenia na którym proces nauki modelu zostanie wykonany</li>
    <li>Konwersja train na tensor_train i umieszczenie go w pamięci karty graficznej</li>
    <li>Zaprojektowanie architektury sieci neuronowej</li>
    <li>Proces nauki</li>
    <li>Zapisanie modelu<li>
</ol>

### Potrzebne importy

In [ ]:
import torch
from torch import nn, optim, randperm

### 1. Wybór karty graficznej do nauki modelu

In [ ]:
device = (
    'cuda'
    if torch.cuda.is_available()
    else 'mps'
    if torch.backends.mps.is_available()
    else 'cpu'
)

### 2. Przeniesienie train do pamięci karty graficznej z podziałem na zbiór treningowy i testowy

In [ ]:
tensor_train = torch.tensor(train, device=device)

### 3. Architektura sieci neuronowej

In [ ]:
model = nn.Sequential(
    nn.Linear(tensor_train.shape[0], 64),
    nn.ReLU(),
    nn.Linear(64, 128),
    nn.ReLU(),
    nn.Linear(128, 72),
    nn.ReLU(),
    nn.Linear(72, 1),
).to(device)

loss_fn = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

### 4. Nauka sieci neuronowej

In [ ]:
n_epochs = 100  # przypomniec sobie
batch_size = 10

for epoch in range(n_epochs):
    tensor_train = tensor_train[randperm(tensor_train.shape[0]), :]
    loss_value = 0.0
    for i, data in enumerate(tensor_train, 0):
        optimizer.zero_grad()
        
        outputs = model(data)
        
        loss = loss_fn(outputs, loss_value)
        
        loss.backward()
        
        optimizer.step()
        
        loss_value += loss.item()
        if i % 10 == 0:
          print('Loss after mini-batch %5d: %.3f' %
                (i + 1, loss_value / 500))
          loss_value = 0.0